# Data-driven modeling - Tutorial 3 - Physics-informed neural networks

Lecturer: Dominik Klein

## Exercise

Create a Tensorflow model for the uniaxial deformation mode of the BCC cell. Use three layers with 16 nodes in each layer, with softplus activation functions. Use two network architectures:
1. "Simple" neural network without additional physical information. Input: {F11, F22, F33}, output: {P11, P22, P33}
2. Physics-informed neural network: model the hyperelastic potential, use $\boldsymbol{P}=\partial_{\boldsymbol{F}}W$ to calculate the stress

## 1. "Simple" neural network

In [ ]:
# Import modules
import numpy  as np
import tensorflow as tf
import matplotlib.pyplot as plt

### Import data

In [ ]:
# Load data for uniaxial deformation case of cubic 'BCC' cell
data = np.loadtxt(f'BCC_uniaxial.txt')

print(data.shape)

Fs = np.array((data[:,0], data[:,4], data[:,8])).transpose()
Ps = np.array((data[:,9], data[:,13],data[:,17])).transpose()

Fs = tf.constant(Fs, dtype = tf.float32)
Ps = tf.constant(Ps, dtype = tf.float32)

# Plot data
plt.plot(Fs[:,0], Fs[:,0], linestyle='--', label='$F_{11}$')
plt.plot(Fs[:,0], Fs[:,1], linestyle='--', label='$F_{22}$')
plt.xlabel('$F_{11}$')
plt.ylabel('$F_{ij}$')
plt.legend()
plt.show()

plt.plot(Fs[:,0], Ps[:,0], linestyle='--', label='$P_{11}$')
plt.plot(Fs[:,0], Ps[:,1], linestyle='--', label='$P_{22}$')
plt.xlabel('$F_{11}$')
plt.ylabel('$P_{ij}$')
plt.legend()
plt.show()

### Define model

In [ ]:
# Create and compile TensorFlow model

model1 = tf.keras.Sequential([
    tf.keras.layers.Dense(16, 'softplus', input_shape=[3]),
    tf.keras.layers.Dense(16, 'softplus'),
    tf.keras.layers.Dense(16, 'softplus'),
    tf.keras.layers.Dense(3)
    ])

         
model1.summary()

model1.compile('adam', 'mse')

# Evaluation for randomly initialized weights and biases
model1.evaluate(Fs, Ps)

### Calibrate and evaluate model

In [ ]:
# Calibrate model
h= model1.fit(Fs, Ps, epochs=100, verbose=2)

# Evaluate and plot calibrated model
Ps_model = model1.predict(Fs)

plt.plot(Fs[:,0], Ps[:,0], linestyle='--', label='$P_{11}$ data')
plt.plot(Fs[:,0], Ps[:,1], linestyle='--', label='$P_{22}$ data')
plt.plot(Fs[:,0], Ps_model[:,0], label='$P_{11}$ model')
plt.plot(Fs[:,0], Ps_model[:,1], label='$P_{22}$ model')
plt.xlabel('$F_{11}$')
plt.ylabel('$P_{ij}$')
plt.legend()
plt.show()

## 2. Physics-informed neural network

### Define model

In [ ]:
# Create potential based constitutive model

def WP(**kwargs):
    Fs = tf.keras.Input(shape=[3])
    Ps = _F_to_P(**kwargs)(Fs)
    model = tf.keras.Model(Fs, Ps)
    model.compile('adam', 'mse')
    return model


# Internal layer, evaluation of PK1 stress with TF GradientTape
class _F_to_P(tf.keras.layers.Layer):
    
    def __init__(self, **kwargs):
        super(_F_to_P, self).__init__()
        self.W = _F_to_W(**kwargs)
        
    def call(self, Fs):
        with tf.GradientTape() as tape:
            tape.watch(Fs)
            Ws = self.W(Fs)
        Ps = tape.gradient(Ws, Fs)
        return Ps
    
    
# Trainable layers for potential    
class _F_to_W(tf.keras.layers.Layer):
    
    def __init__(self, ns = [16, 16, 16]):
        super(_F_to_W, self).__init__()
        self.l0 = tf.keras.layers.Dense(ns[0], 'softplus', use_bias=True)
        self.ls = [tf.keras.layers.Dense(n, 'softplus', \
                        use_bias=True, \
                        kernel_constraint=tf.keras.constraints.non_neg()) \
                   for n in ns[1:]]
        self.ls.append(tf.keras.layers.Dense(1, \
                        use_bias=True, \
                        kernel_constraint=tf.keras.constraints.non_neg()))

        
    def W_core(self, Fs):
        
        x = self.l0(Fs)
            
        for l in self.ls:
            x = l(x)
                
        return x
        
    def call(self, Fs):
        return self.W_core(Fs)
    
    
model2=WP()

model2.summary()

### Calibrate and evaluate model

In [ ]:
# Calibrate model
h= model2.fit(Fs, Ps, epochs=100, verbose=2)

# Evaluate and plot calibrated model
Ps_model_2 = model2.predict(Fs)

plt.plot(Fs[:,0], Ps[:,0], linestyle='--', label='$P_{11}$ data')
plt.plot(Fs[:,0], Ps[:,1], linestyle='--', label='$P_{22}$ data')
plt.plot(Fs[:,0], Ps_model_2[:,0], label='$P_{11}$ model')
plt.plot(Fs[:,0], Ps_model_2[:,1], label='$P_{22}$ model')
plt.xlabel('$F_{11}$')
plt.ylabel('$P_{ij}$')
plt.legend()
plt.show()